In [1069]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np

%matplotlib inline
import plotly.graph_objects as go

In [1070]:
dataPath_Raw = ("../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
dataPath_Processed = ("../data/processed/")

pd.set_option("display.max_rows", 500)

mpl.rcParams['figure.figsize'] = (20,16)
pd.set_option('display.max_rows', 500)

In [1071]:
df_analyse = pd.read_csv(dataPath_Processed + "COVID_Flat_Table.csv", sep=",",
                        parse_dates=[0])

df_analyse.sort_values('date',ascending=True).tail()



,date,Italy,US,Spain,Germany,India,"Korea, South",China,Brazil
136,2020-06-06,234801,1925765,241310,185450,246622,11776,84186,672846
137,2020-06-07,234998,1943882,241550,185750,257486,11814,84191,691758
138,2020-06-08,235278,1961428,241717,186109,265928,11852,84195,707412
139,2020-06-09,235561,1979489,241966,186506,276146,11902,84198,739503
140,2020-06-10,235763,2000464,242280,186522,276583,11947,84209,772416


In [1072]:
country_list = [
    'Italy',
    'US',
    'Spain',
    'Germany',
    'India',
    'Korea, South',
    'China',
    'Brazil'
]

## Helper Functions

In [1073]:
def quick_plot(x_in, df_input, yscale='log', slider=False):
    fig = go.Figure()
    
    for i in df_input.columns:
        fig.add_trace(go.Scatter(x = x_in,
                         y = df_input[i],
                         mode = 'lines + markers',
                         line_width = 0.4, marker_size = 3,
                         name = i)
                  )

    fig.update_layout(autosize = True,
              width = 1024,
              height = 780,
              font = dict(family = 'PT Sans. monospace',
                          size = 18,
                          color = '#7f7f7f'
              
                          )
                     )
    fig.update_yaxes(type=yscale),

    fig.update_xaxes(tickangle = -45,
                    nticks = 20,
                    tickfont = dict(size= 14, color = '#7f7f7f')
                    )
    
    if slider == True:

        fig.update_layout(xaxis_rangeslider_visible= True)
        
    fig.show()
    
    



In [1074]:
#data = pd_sync_timelines_W_slope

#for columns in data:
 #   print(data['Italy'])
    
#df1 = data.iloc[:,4:-1]
#df1

def quick_plot_new(x_in, df_input, yscale='log', slider=False):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x = x_in,
                             y = df_input['doubling every day'], 
                             line_color = "black", opacity=0.3, marker_size = 7,
                             mode = 'markers', name='doubling every day'))
    fig.add_trace(go.Scatter(x = x_in,
                             y = df_input['doubling every two days'], 
                             line_color = "black", opacity=0.3, marker_size = 7,
                             mode = 'markers', name='doubling every two days'))

    fig.add_trace(go.Scatter(x = x_in,
                             y = df_input['doubling every 4 days'], 
                             line_color = "black", opacity=0.3, marker_size = 7,
                             mode = 'markers', name='doubling every 4 days'))
    fig.add_trace(go.Scatter(x = x_in,
                             y = df_input['doubling every 10 days'], 
                             line_color = "black", opacity=0.3, marker_size = 7,
                             mode = 'markers', name='doubling every 10 days'))

    for i in df_input.columns[4:-1]:
        fig.add_trace(go.Scatter(x = x_in,
                         y = df_input[i],
                         mode = 'lines',
                         line_width = 2.5, marker_size = 3,
                         name = i)
                  )

    fig.update_layout(autosize = True,
              width = 1024,
              height = 780,
              font = dict(family = 'PT Sans. monospace',
                          size = 18,
                          color = '#7f7f7f'
              
                          ),
              title='CORONA Infection Rate (slope)', title_x=0.5,
              xaxis_title='Time in Days',
              yaxis_title='Confirmed infected people [source: John Hopkins csse, log-scale]'
                     )
    
    fig.update_layout()
    
    fig.update_yaxes(type=yscale, range=[2, 8])

    fig.update_xaxes(tickangle = -45,
                    nticks = 20,
                    tickfont = dict(size= 14, color = '#7f7f7f')
                    )
    
    if slider == True:

        fig.update_layout(xaxis_rangeslider_visible= True)
        
    fig.show()

In [1075]:
quick_plot(df_analyse.date,
          df_analyse.iloc[:,1:],
          yscale= 'linear',
          slider=True
          )

In [1076]:
threshold = 100

In [1077]:
compare_list = []

for pos, country in enumerate(country_list):
    compare_list.append(np.array(df_analyse[country][df_analyse[country]>threshold]))

In [1078]:
pd_sync_timelines = pd.DataFrame(compare_list, index=df_analyse.columns[1:]).T

In [1079]:
pd_sync_timelines['date']=np.arange(pd_sync_timelines.shape[0])

In [1080]:
quick_plot(pd_sync_timelines.date,
          pd_sync_timelines.iloc[:,:-1],
          yscale= 'log',
          slider=True
          )

### Doubling Rate

$N(t)=N_0*2^{t/T}$

In [1081]:
def doubling_rate(N_0, t, T_d):
    return N_0*np.power(2, t/T_d)

In [1082]:
max_days = 140

norm_slope = {
    
    'doubling every day': doubling_rate(100, np.arange(max_days),1),
    'doubling every two days': doubling_rate(100, np.arange(max_days),2),
    'doubling every 4 days': doubling_rate(100, np.arange(max_days),4),
    'doubling every 10 days': doubling_rate(100, np.arange(max_days),10)
}


In [1083]:
pd_sync_timelines_W_slope = pd.concat([pd.DataFrame(norm_slope), pd_sync_timelines], axis=1)
pd_sync_timelines_W_slope;

In [1084]:
quick_plot(pd_sync_timelines_W_slope.date,
          pd_sync_timelines_W_slope.iloc[:,:-1],
          yscale= 'log',
          slider=True
          )


In [1085]:
quick_plot_new(pd_sync_timelines_W_slope.date,
          pd_sync_timelines_W_slope.iloc[:,:-1],
          yscale= 'log',
          slider=True
          )